In [1]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("NFL Play by Play 2009-2018 (v5).csv")

df.head()

C:\Users\gabri\AppData\Local\Temp\ipykernel_9396\638845504.py:4: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("NFL Play by Play 2009-2018 (v5).csv")


,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,46,2009091000,PIT,TEN,PIT,home,TEN,TEN,30.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
1,68,2009091000,PIT,TEN,PIT,home,TEN,PIT,58.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
2,92,2009091000,PIT,TEN,PIT,home,TEN,PIT,53.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
3,113,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
4,139,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0


In [2]:
r_off_agg = df[(df.play_type == 'run')]
p_off_agg = df[(df.play_type == 'pass')|(df.play_type == 'sack')]

r_off_agg = r_off_agg.groupby(['game_id','qtr','posteam'])['yards_gained'].mean().reset_index()
p_off_agg = p_off_agg.groupby(['game_id','qtr','posteam'])['yards_gained'].mean().reset_index()

r_off_agg = r_off_agg.rename(columns={'yards_gained':'RushingMean'})
p_off_agg = p_off_agg.rename(columns={'yards_gained':'PassingMean'})

off_agg = pd.merge(r_off_agg, p_off_agg, left_on=['game_id','qtr','posteam'], right_on=['game_id','qtr','posteam'], how='outer')

off_agg.head()



,game_id,qtr,posteam,RushingMean,PassingMean
0,2009091000,1,PIT,0.333333,-0.125000
1,2009091000,1,TEN,6.166667,6.222222
2,2009091000,2,PIT,2.500000,9.062500
3,2009091000,2,TEN,1.400000,8.307692
4,2009091000,3,PIT,2.000000,3.000000


In [5]:
df = df.merge(off_agg)

In [6]:
df = df[["qtr", "yardline_100", "ydstogo", "half_seconds_remaining","down","score_differential","posteam","posteam_type","play_type","game_seconds_remaining","posteam_timeouts_remaining","PassingMean","RushingMean"]]
df.head()

,qtr,yardline_100,ydstogo,half_seconds_remaining,down,score_differential,posteam,posteam_type,play_type,game_seconds_remaining,posteam_timeouts_remaining,PassingMean,RushingMean
0,1,30.0,0,1800.0,NaN,NaN,PIT,home,kickoff,3600.0,3.0,-0.125,0.333333
1,1,58.0,10,1793.0,1.0,0.0,PIT,home,pass,3593.0,3.0,-0.125,0.333333
2,1,53.0,5,1756.0,2.0,0.0,PIT,home,run,3556.0,3.0,-0.125,0.333333
3,1,56.0,8,1715.0,3.0,0.0,PIT,home,pass,3515.0,3.0,-0.125,0.333333
4,1,56.0,8,1707.0,4.0,0.0,PIT,home,punt,3507.0,3.0,-0.125,0.333333


In [7]:
df = df.dropna()
df.isnull().sum()

qtr                           0
yardline_100                  0
ydstogo                       0
half_seconds_remaining        0
down                          0
score_differential            0
posteam                       0
posteam_type                  0
play_type                     0
game_seconds_remaining        0
posteam_timeouts_remaining    0
PassingMean                   0
RushingMean                   0
dtype: int64

In [8]:
df.nunique()

qtr                              5
yardline_100                    99
ydstogo                         47
half_seconds_remaining        1801
down                             4
score_differential             109
posteam                         35
posteam_type                     2
play_type                        7
game_seconds_remaining        3601
posteam_timeouts_remaining       4
PassingMean                   1780
RushingMean                    877
dtype: int64

In [9]:
def shorten_categories(categories, cutoff):
    categorical_map = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff:
            categorical_map[categories.index[i]] = categories.index[i]
        #else:
            #categorical_map[categories.index[i]] = 'Other'
    return categorical_map

In [10]:
country_map = shorten_categories(df.play_type.value_counts(), 100000)
df['play_type'] = df['play_type'].map(country_map)
df.play_type.value_counts()

pass    183949
run     131843
Name: play_type, dtype: int64

In [11]:
df.nunique()

qtr                              5
yardline_100                    99
ydstogo                         47
half_seconds_remaining        1801
down                             4
score_differential             109
posteam                         35
posteam_type                     2
play_type                        2
game_seconds_remaining        3601
posteam_timeouts_remaining       4
PassingMean                   1780
RushingMean                    877
dtype: int64

In [14]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 315792 entries, 1 to 436121
Data columns (total 13 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   qtr                         315792 non-null  int64  
 1   yardline_100                315792 non-null  float64
 2   ydstogo                     315792 non-null  int64  
 3   half_seconds_remaining      315792 non-null  float64
 4   down                        315792 non-null  float64
 5   score_differential          315792 non-null  float64
 6   posteam                     315792 non-null  object 
 7   posteam_type                315792 non-null  object 
 8   play_type                   315792 non-null  object 
 9   game_seconds_remaining      315792 non-null  float64
 10  posteam_timeouts_remaining  315792 non-null  float64
 11  PassingMean                 315792 non-null  float64
 12  RushingMean                 315792 non-null  float64
dtypes: float64(8),

In [15]:
df = df.dropna()
df.isnull().sum()

qtr                           0
yardline_100                  0
ydstogo                       0
half_seconds_remaining        0
down                          0
score_differential            0
posteam                       0
posteam_type                  0
play_type                     0
game_seconds_remaining        0
posteam_timeouts_remaining    0
PassingMean                   0
RushingMean                   0
dtype: int64

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315792 entries, 1 to 436121
Data columns (total 13 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   qtr                         315792 non-null  int64  
 1   yardline_100                315792 non-null  float64
 2   ydstogo                     315792 non-null  int64  
 3   half_seconds_remaining      315792 non-null  float64
 4   down                        315792 non-null  float64
 5   score_differential          315792 non-null  float64
 6   posteam                     315792 non-null  object 
 7   posteam_type                315792 non-null  object 
 8   play_type                   315792 non-null  object 
 9   game_seconds_remaining      315792 non-null  float64
 10  posteam_timeouts_remaining  315792 non-null  float64
 11  PassingMean                 315792 non-null  float64
 12  RushingMean                 315792 non-null  float64
dtypes: float64(8),

In [17]:
df.head()

,qtr,yardline_100,ydstogo,half_seconds_remaining,down,score_differential,posteam,posteam_type,play_type,game_seconds_remaining,posteam_timeouts_remaining,PassingMean,RushingMean
1,1,58.0,10,1793.0,1.0,0.0,PIT,home,pass,3593.0,3.0,-0.125,0.333333
2,1,53.0,5,1756.0,2.0,0.0,PIT,home,run,3556.0,3.0,-0.125,0.333333
3,1,56.0,8,1715.0,3.0,0.0,PIT,home,pass,3515.0,3.0,-0.125,0.333333
5,1,43.0,10,1584.0,1.0,0.0,PIT,home,pass,3384.0,3.0,-0.125,0.333333
6,1,40.0,7,1548.0,2.0,0.0,PIT,home,pass,3348.0,3.0,-0.125,0.333333


In [18]:
from sklearn.preprocessing import LabelEncoder
le_posteam = LabelEncoder()
df['posteam'] = le_posteam.fit_transform(df['posteam'])
df["posteam"].unique()

array([27, 33,  7, 20, 22, 10,  8, 32, 12, 24, 13, 14,  9,  6,  4, 26, 16,
        2,  1, 19,  0, 30, 31, 29, 23, 34, 11,  5, 21,  3, 25, 28, 17, 15,
       18])

In [19]:
le_posteam_type = LabelEncoder()
df['posteam_type'] = le_posteam_type.fit_transform(df['posteam_type'])
df["posteam_type"].unique()

array([1, 0])

In [20]:
le_play_type = LabelEncoder()
df['play_type'] = le_play_type.fit_transform(df['play_type'])
df["play_type"].unique()

array([0, 1])

In [ ]:
#gbc = GradientBoostingClassifier()
#gbc.fit(train_X, train_y)

#gbc_predictions = gbc.predict(val_X)

#false_positive_rate, true_positive_rate, thresholds = roc_curve(val_y, gbc_predictions)
#gbc_roc_auc = auc(false_positive_rate, true_positive_rate)